In [ ]:
# Import modules from ase and fireworks
import os
import numpy as np

from ase.build import bulk
from ase.calculators.emt import EMT
from ase.db.core import connect

from fireworks import Firework, FWorker, LaunchPad, PyTask, FWAction, Workflow
from fireworks.core.rocket_launcher import launch_rocket, rapidfire

# Minimal hilde inputs to make dictionary conversion easier
from hilde.structure.structure import pAtoms, dict2patoms, patoms2dict
from hilde.helpers.hash import hash_atoms

mod_name = __name__

In [ ]:
# Define a single point calculation calculate wrapper
def calculate(atoms_dict):
    at = dict2patoms(atoms_dict)
    at.calc.calculate(at, properties=['forces'])
    # update_spec modifies the spec of the current Firework's children to include new information with
    # the new/updated spec keys set to the keys in the dict
    return FWAction(update_spec={"calc_atoms": patoms2dict(at)})

In [ ]:
# Intialize Structures
ni = pAtoms(bulk('Ni', cubic=True))
ni.set_calculator(EMT())
ni_dict = patoms2dict(ni)

ni_hash, calc_hash = hash_atoms(ni)

In [ ]:
# To pass things to Fireworks they need to be in json string form or an object that can be easily
# converted to a json string (like a dict)
# Function are passed by their module_name.file_name
ft_calc = PyTask({"func": mod_name+".calculate", "args": [ni_dict]})

In [ ]:
# Add a firework with the task to define a job
wf = Workflow([Firework([ft_calc])])
launchpad = LaunchPad()
launchpad.add_wf(wf)
# launch_rocket computes a single Firework
launch_rocket(launchpad, strm_lvl="CRITICAL")

In [ ]:
# Once something is inside Fireworks it can only be accessed through it's database or a database/file
# the user specified inside the tasks
# Getting the results from FireWorks
fw_spec = launchpad.get_fw_dict_by_id(wf.root_fw_ids[0])
forces = fw_spec['launches'][0]['action']['update_spec']['calc_atoms']['forces']
print(f"Atomic forces are:\n{np.array(forces)}")

In [ ]:
# Define a function that will add a new structure calculated within Fireworks to an ase databse
def calc_to_db(db_path, atoms_dict):
    db = connect(db_path)
    at = dict2patoms(atoms_dict)
    at.calc.atoms = at
    atoms_hash, calc_hash = hash_atoms(at)
    selection = [("atoms_hash", "=", atoms_hash), ("calc_hash", "=", calc_hash)]
    # Try to update the database if the material is already present, if not add it to the database
    try:
        rows = list(db.select(selection=selection))
        if not rows:
            raise KeyError()
        for row in rows:
            db.update(row.id, at, atoms_hash=atoms_hash, calc_hash=calc_hash)
    except KeyError:
        db.write(at, atoms_hash=atoms_hash, calc_hash=calc_hash)
    return FWAction()

In [ ]:
# Set up database
db_path = (os.getcwd() + '/test.db')
print(f'database: {db_path}')
# Set up a Workflow where each FireTask has its own Firework
fw_calc = Firework(PyTask({"func": mod_name+".calculate", "args": [ni_dict]}))
# Args and Inputs are broken up into two objects (Can be up to three) 
#     args: Positional arguments that are given to Fireworks from outside its current spec
#     inputs: Postiional arguments that are given to Fireworks from inside its current spec
#             pased as a string to the spec key
#     kwargs: dict describing keyword arguments taken from outside current spec (none in this example)
#     Function call fxn_name(args, inputs, kwargs)
fw_to_db = Firework(PyTask({"func":mod_name+".calc_to_db", "args":[db_path], "inputs":["calc_atoms"]}))
# Workflows defined by list of Fireworks, and a dict describing the links between each Firework
wf = Workflow([fw_calc, fw_to_db], {fw_calc:[fw_to_db]})
launchpad.add_wf(wf)
# Launch 2 rockets to complete the workflow
rapidfire(launchpad, nlaunches=2, strm_lvl="CRITICAL")

In [ ]:
# See the results with the database access
db = connect(db_path)
at = db.get_atoms(calc_hash=calc_hash, atoms_hash=ni_hash, attach_calculator=True)
row = list(db.select(selection=[("atoms_hash", "=", ni_hash)], columns=["forces"]))[0]
print(f"Atomic forces from the new atoms are: \n{at.get_forces()}")
print(f"Atomic forces from the row is: \n{row.forces}")

In [ ]:
# Combined local/remote queue launching
from hilde.fireworks_api_adapter.combined_launcher import rapidfire as lq_rapidfire
from hilde.fireworks_api_adapter.launchpad import LaunchPadHilde as LaunchPad
# Import the hilde calculate function so both the local and remote machines have the same function in their path
from hilde.tasks.fireworks import calculate as hilde_calc

In [ ]:
#Port changes are for my setup
launchpad = LaunchPad(port=27018, strm_lvl="CRITICAL")

In [ ]:
# Redefine calc_to_db to match the hilde calc
def calc_to_db(db_path, atoms_dict):
    db = connect(db_path)
    at = dict2patoms(atoms_dict[0])
    at.calc.atoms = at
    atoms_hash, calc_hash = hash_atoms(at)
    selection = [("atoms_hash", "=", atoms_hash), ("calc_hash", "=", calc_hash)]
    # Try to update the database if the material is already present, if not add it to the database
    try:
        rows = list(db.select(selection=selection))
        if not rows:
            raise KeyError()
        for row in rows:
            db.update(row.id, at, atoms_hash=atoms_hash, calc_hash=calc_hash)
    except KeyError:
        db.write(at, atoms_hash=atoms_hash, calc_hash=calc_hash)
    return FWAction()

In [ ]:
q_spec = {
    # Submission script changes are controled by the _queueadapter dictionary
    "_queueadapter":{
        # Keys are the same that you define in "my_qadapter.yaml"
        "walltime": "00:01:00",
        "nodes": 1
    }
}

In [ ]:
# Remote Settings (Change these to match what you need)
remote_settings={
    # Has to be a full path to the host, draco.mpcdf.mpg.de will time out. Needs to be a list
    "remote_host": ["draco01.mpcdf.mpg.de"],
    # Not necessary if remote host is username@host
    "remote_user": "tpurcell",
    # Try to avoid using this
    "remote_password": None,
    "remote_config_dir": ["/u/tpurcell/.fireworks"],
    # If you are using Kerberos this needs to be True
    "gss_auth": True
}

In [ ]:
# Set up a Workflow where each FireTask has its own Firework
wd = "/u/tpurcell/.fireworks/Ni/"
fw_calc = Firework(PyTask({"func": hilde_calc.name, "args": [wd, "calc_atoms", ni_dict]}), spec=q_spec)
fw_to_db = Firework(PyTask({"func":mod_name+".calc_to_db", "args":[db_path], "inputs":["calc_atoms"]}))
# Workflows defined by list of Fireworks, and a dict describing the links between each Firework
wf = Workflow([fw_calc, fw_to_db], {fw_calc:[fw_to_db]})

In [ ]:
launchpad.add_wf(wf)
lq_rapidfire(launchpad, 
             launch_dir='.', 
             nlaunches=0, 
             njobs_queue=250, 
             wflow=wf, 
             njobs_block=500,
             sleep_time=60, 
             reserve=True, 
             remote_host=remote_settings['remote_host'],
             remote_user=remote_settings['remote_user'], 
             remote_password=remote_settings['remote_password'],
             remote_config_dir=remote_settings["remote_config_dir"],
             # List of tasks that should be done on the queue
             tasks2queue=[hilde_calc.name],
             gss_auth=remote_settings["gss_auth"])

In [ ]:
# See the results with the database access
db = connect(db_path)
at = db.get_atoms(calc_hash=calc_hash, atoms_hash=ni_hash, attach_calculator=True)
row = list(db.select(selection=[("atoms_hash", "=", ni_hash)], columns=["forces"]))[0]
print(f"Atomic forces from the new atoms are: \n{at.get_forces()}")
print(f"Atomic forces from the row is: \n{row.forces}")